# Exporting to CSV
After collecting data in your program, it's often useful to save it in an external file for easier sharing and analysis. CSV (Comma-Separated Values) is a widely used format because it is simple, lightweight, and compatible with many applications, including spreadsheets and databases. You can read more about the format [here](https://docs.fileformat.com/spreadsheet/csv/). 

The following script demonstrates how to export data programmatically from an object into a CSV file.

In [12]:
import os
import pathlib

example_user_object = {
    "type": "user",
    "username": "JohnDoe123",
    "id": 1,
    "first_name": "John",
    "last_name": "Doe",
    "message_count": 23
}

example_message_object = {
    "type": "message",
    "sender": example_user_object,
    "content": "This is the message",
    "media": True,
    "media_download": "https://link_to_media.com/media?q=abcdefg"
}

example_object_list = [
    example_user_object,
    example_message_object
]

with open("example_users_csv.csv", "a") as users_file, open("example_messages_csv.csv", "a") as messages_file:

    #iterate through example list of objects
    for obj in example_object_list:

        #handle user objects
        if obj["type"] == "user":
            #if the file is empty, write the first row of labels
            if os.path.getsize("example_users_csv.csv") == 0:
                label_row = ""
                for key, value in obj.items():
                    label_row += key + ","
                label_row = label_row[:-1] + "\n"
                users_file.writelines(label_row)
                

            row = ""
            for key, value in obj.items():
                row += str(value) + ","
            row = row[:-1] + "\n"
            users_file.writelines(row)

        #handle message objects
        if obj["type"] == "message":
            #if the file is empty, write the first row of labels
            if os.path.getsize("example_messages_csv.csv") == 0:
                    label_row = ""
                    for key, value in obj.items():
                        if key == "sender":
                            label_row += "sender_id" + ","
                        else:
                            label_row += key + ","
                    label_row = label_row[:-1] + "\n"
                    messages_file.writelines(label_row)
                    

            row = ""
            for key, value in obj.items():
                if type(value) is dict:
                    row += str(value["id"]) + ","
                else:
                    row += str(value) + ","
            row = row[:-1] + "\n"
            messages_file.writelines(row)

#printing the contents of the files for demonstration purposes
print("messages file:")
with open("example_messages_csv.csv", "r") as file:
    for line in file:
        print(line, end="")

print("\nusers file:")
with open("example_users_csv.csv", "r") as file:
    for line in file:
        print(line, end="")

file_to_rem = pathlib.Path("example_users_csv.csv")
file_to_rem.unlink()
file_to_rem = pathlib.Path("example_messages_csv.csv")
file_to_rem.unlink()

messages file:
type,sender_id,content,media,media_download
message,1,This is the message,True,https://link_to_media.com/media?q=abcdefg

users file:
type,username,id,first_name,last_name,message_count
user,JohnDoe123,1,John,Doe,23


The next script takes it a step further, getting the last 10 messages from the nytimes Telegram channel and writing its data to a csv file.

In [1]:
from dotenv import load_dotenv
from telethon import TelegramClient
import os
import pathlib
import datetime

load_dotenv()

TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
PHONE_NUMBER = os.getenv("PHONE_NUMBER")

async with TelegramClient("simpatbos", TELEGRAM_API_ID, TELEGRAM_API_HASH) as client:
    channel_name = "nytimes"
    limit = 10
    with open("nytimes_posts.csv", "a") as file:
        written_labels = False
        async for message in client.iter_messages(channel_name, limit):
            message = message.to_dict()
            if not written_labels:
                label_row = ""
                for key in message.keys():
                    if key == "_":
                        label_row += "type,"
                    else:
                        label_row += key + ","
                label_row = label_row[:-1] + "\n"
                file.writelines(label_row)
                written_labels = True
                
    
            row = ""
            for value in message.values():
                if type(value) is not str and type(value) is not int and type(value) is not bool and type(value) is not list and value is not None:
                    if type(value) is datetime.datetime:
                        row += value.strftime("%Y-%m-%d %H:%M:%S")
                    else:
                        row += value["_"] + " Object"
                else:
                    row += str(value)
                    row = row.replace("\n", "\\n")
                    row = row.replace(",", ";")
                row += ","
            row = row[:-1] + "\n"
            file.writelines(row)

#printing the contents of the file for demonstration purposes
with open("nytimes_posts.csv", "r") as file:
    for line in file:
        print(line, end="")

file_to_rem = pathlib.Path("nytimes_posts.csv")
file_to_rem.unlink()

type,id,peer_id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,legacy,edit_hide,pinned,noforwards,invert_media,offline,video_processing_pending,from_id,from_boosts_applied,saved_peer_id,fwd_from,via_bot_id,via_business_bot_id,reply_to,media,reply_markup,entities,views,forwards,replies,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period,quick_reply_shortcut_id,effect,factcheck
Message;3376;PeerChannel Object;2025-04-03 18:55:26;Here are some of the stories we are covering from around the world:\n\nThe War on Nature in Ukraine\n\nThe human costs of Russia’s war in Ukraine are enormous. But Ukraine’s environment is also being devastated. The war may end; but damage from artillery shells; mines; drones and missiles will endure for decades; experts say.\n\nWhere Trump’s Tariffs Will Hit Hardest\n\nThese countries are some of the most vulnerable to Washington’s sweeping new tariffs. Their economies are deeply reliant on selling goods to the United Sta